In [1]:
!pip install -U langchain-pinecone


In [2]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [3]:
pc = Pinecone(api_key="pcsk_8vYED_L33foiXxgL8YQd7oaFecQdq28MARigPJGt7BbuyGeBDE19Rvug695pYqJr9t3Yd")
index = pc.Index("us-places-rag2")

In [4]:
# 02:10:59

In [5]:
!pip install langchain_community

In [6]:
pip install -qU langchain-huggingface

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

In [11]:
# embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # 384 dims

embed_model = embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
text_field = "text"
# vectorstore = Pinecone(
#     index,embed_model.embed_query,text_field
# )
# Create vector store wrapper
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embed_model,    # pass full embedding object, not .embed_query
    text_key="text"
)

In [13]:
query = "What is Vizcaya Museum and Gardens"
vectorstore.similarity_search(query, k = 1)

[Document(id='Miami_Vizcaya_Museum_and_Gardens_0', metadata={'categories': ['Speciality Museums'], 'location': 'Miami', 'name': 'Vizcaya Museum and Gardens', 'rating': 4.5, 'reviews': ['The Hearst Castle of the East, When I first saw Vizcaya Museum and Gardens in Miami, Florida, even before I began my tour of the extraordinary 70-room villa and estate built by businessman James Deering of the Deering McCormick International Harvester fortune, it reminded me of American circus mogul John Ringling\'s Ca\' d\'zan in Sarasota, Florida. Located on Biscayne Bay in the present day neighborhood of Coconut Grove, the 50-acre complex was constructed between 1914 and 1923 at a cost of $15 million. It includes the villa, extensive Italian Renaissance gardens, native woodland landscape and historic village outbuildings. The villa contains rooms of distinctive architectural interiors decorated with numerous antiques, with an emphasis on 15th through early 19th century European decorative art and fur

In [19]:
def augment_prompt(query:str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query

    Contexts:
    {source_knowledge}

    Query : {query}"""
    return augmented_prompt

In [20]:
print(augment_prompt(query))

Using the contexts below, answer the query

    Contexts:
    Built in 1916 as a winter retreat, this lavish villa is a tribute to the Italian Renaissance. The museum contains much of the original furnishings and artwork, and is surrounded by lush, formal gardens.
The museum consists of eleven stunningly sculptured life-size dioramas. It combines India’s ancient doll-making artistry and yogic wisdom with America’s advanced multimedia tech. Different concepts of the Bhagavad-gita, renowned as the jewel of India’s yogic knowledge, are introduced in the course of 45 minutes of narrative accompanied by animatronics, light show and video projection which leaves the audience with a magical experience.
A Venetian palace in the middle of Boston, Gardner's home is now a museum displaying her impressive, eclectic collection of European, American and Asian art, including sculpture, paintings, furniture, ceramics and textiles. Visitors can stroll or rest in a spectacular skylit courtyard filled wi

In [24]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

In [25]:
messages = [
    SystemMessage(content = "You are a helpful assistant"),
    HumanMessage(content = "Hi AI, how are you today?"),
    AIMessage(content = "I'm great thank you. How can I help you?"),
    HumanMessage(content = "I'd like to understand string theory")
]